## Indexing of large text collection 

In the vector space Information Retrieval (IR) model, a vector is used to represent each item or document in a collection. Each component
of the vector reflects a particular concept, key word, or term associated with
the given document. The value assigned to that component reflects the importance of
the term in representing the semantics of the document. Typically, the value is a function
of the frequency with which the term occurs in the document or in the document
collection as a whole.

A simple collection of five titles is 
described by six terms leads to a 6*5 term-by-document matrix, the matrix elements in this example
are scaled so that the Euclidean norm of each column is 1.

In constructing a term-by-document matrix, terms are usually identified by their
word stems. The word pastries counts as the term pastry, and
the word baking counts as the term bake.

The t = 6 terms:

T1: bak(e,ing)

T2: recipes

T3: bread

T4: cake

T5: pastr(y,ies)

T6: pie

The d = 5 document titles:

D1: How to {Bake} {Bread} Without {Recipes}

D2: The Classic Art of Viennese {Pastry}

D3: Numerical {Recipes}: The Art of Scientific Computing

D4: {Breads}, {Pastries}, {Pies} and {Cakes}: Quantity {Baking} {Recipes}

D5: {Pastry}: A Book of Best French {Recipes}

In [15]:
import numpy as np
A = np.array([[1,0,0,1,0],
              [1,0,1,1,1],
              [1,0,0,1,0],
              [0,0,0,1,0],
              [0,1,0,1,1],
              [0,0,0,1,0] ])

In [16]:
(m,n)=A.shape
eu=np.array(np.zeros(n))
for i in range(n):
    eu[i] = np.linalg.norm(A[:,i],2)

In [18]:
As1= np.dot(A,np.diag(1/eu))
As = A/eu
As1-As

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### Query matching based on angles in a six-dimensional vector space

Suppose that a user in search of cooking information initiates a search for books about baking
bread. The query vector is:

In [20]:
query1 = np.array([ 1,0,1,0,0,0])
print(query1.shape)
query1 = query1.reshape(m,1)
print(query1.shape)

(6,)
(6, 1)


The search for relevant documents
is carried out by computing the cosines of the angles j between the query
vector and the document vectors. A document is returned as relevant
only if the cosine of the angle it makes with the query vector is greater than some
threshold or cutoff value.

In [21]:
# cos of the angles  cos(teta) = x^T y / (||x||*||y||)
np.dot(query1.T,As)/np.linalg.norm(query1,2) # the vector contains the inner product query1.T*As_*1 (first column), query1.T*As_*2
# if the threshold is 0.5 the two documents related to this query  are D1,D4

array([[0.81649658, 0.        , 0.        , 0.57735027, 0.        ]])

question: requested books about baking

In [22]:
query2 =  np.array([ 1,0,0,0,0,0])
query2 =  query2.reshape(m,1)
# scaled query vector
#query2=query2/np.linalg.norm(query2,2)

In [23]:
# cos of the angles
np.dot(query2.T,As)/np.linalg.norm((query2),2)

array([[0.57735027, 0.        , 0.        , 0.40824829, 0.        ]])

threshold=0.5 Only the first document, a book about baking bread,
makes the cosine cutoff. The fourth document, which is in fact a more comprehensive
reference about baking, is not returned as relevant.

### Rank reduction

Now let us identify dependence between the columns or rows of the term-by-
document matrix. For a rank rA matrix, the rA basis vectors of its column space
serve in place of its d column vectors to represent its column space. One set of basis
vectors is found by computing the QR factorization of the term-by-document matrix.

In [10]:
np.set_printoptions(suppress=True)
[Q,R]=np.linalg.qr(As)
print('Matrix Q')
print(Q)
print('Matrix R')
print(R)

Matrix Q
[[-0.57735027  0.         -0.40824829 -0.         -0.70710678]
 [-0.57735027  0.          0.81649658 -0.         -0.        ]
 [-0.57735027 -0.         -0.40824829  0.          0.70710678]
 [-0.         -0.          0.         -0.70710678  0.        ]
 [-0.         -1.          0.          0.          0.        ]
 [-0.         -0.          0.         -0.70710678  0.        ]]
Matrix R
[[-1.          0.         -0.57735027 -0.70710678 -0.40824829]
 [ 0.         -1.          0.         -0.40824829 -0.70710678]
 [ 0.          0.          0.81649658 -0.          0.57735027]
 [ 0.          0.          0.         -0.57735027 -0.        ]
 [ 0.          0.          0.          0.         -0.        ]]


In [11]:
import scipy.linalg as spl
[Q,R,P]=spl.qr(As,mode='economic',pivoting=True)
print('Q')
print(Q)
print('R')
print(R)
print('P')
print(P)

Q
[[-0.57735027  0.         -0.40824829 -0.         -0.70710678]
 [-0.57735027  0.          0.81649658 -0.         -0.        ]
 [-0.57735027 -0.         -0.40824829  0.          0.70710678]
 [-0.         -0.          0.         -0.70710678  0.        ]
 [-0.         -1.          0.          0.          0.        ]
 [-0.         -0.          0.         -0.70710678  0.        ]]
R
[[-1.          0.         -0.57735027 -0.70710678 -0.40824829]
 [ 0.         -1.          0.         -0.40824829 -0.70710678]
 [ 0.          0.          0.81649658 -0.          0.57735027]
 [ 0.          0.          0.         -0.57735027 -0.        ]
 [ 0.          0.          0.          0.         -0.        ]]
P
[0 1 2 3 4]


In [12]:
QA=np.copy(Q[:,0:4])
# QA basis for the column space of A
RA=np.copy(R[0:4,:])
# In general, it is necessary to use column 
# pivoting during the QR factorization to ensure that the zeros appear 
# at the bottom of the matrix AP = QR
QAO =np.copy( Q[:,4:6])
# The  columns of QAO are a basis for the orthogonal
# complement of the column space of AP and so of the column space of A.
# Column pivoting provides important numerical advantages without changing the database, as
# permuting the columns of A results only in a reordering of the document vectors.

The semantic content of a database is fully described by any basis for the column
space of the associated term-by-document matrix, and query matching proceeds with
the factors QR in place of the matrix A.

In [13]:
costet=np.dot(As.T,query1)/np.linalg.norm(query1,2)
print(costet)
costet=np.dot(RA.T,np.dot(QA.T,query1))/np.linalg.norm(query1,2)
print(costet)

[[0.81649658]
 [0.        ]
 [0.        ]
 [0.57735027]
 [0.        ]]
[[ 0.81649658]
 [ 0.        ]
 [-0.        ]
 [ 0.57735027]
 [ 0.        ]]


In [14]:
RAs=np.copy(R) 
RAs[3:6,:]=0
E = np.dot(Q,RAs)-np.dot(Q,R)
AE = As+E
AEeu=np.array(np.zeros((1,n)))
for i in range(n):
    AEeu[0,i] = np.linalg.norm(AE[:,i],2)

QAs=np.copy(Q[:,0:3])
# QAs basis for the column space of a subsapce of the columns space A
RAs=RAs[0:3,:]

#(QAs,RAs)=np.linalg.qr(AE)

costeta1=np.dot(RAs.T,np.dot(QAs.T,query1))/(AEeu.T*np.linalg.norm((query1),2))
print('cos query 1')
print(costeta1)

print('cos query 2')
costeta2=np.dot(RAs.T,np.dot(QAs.T,query2))/(AEeu.T*np.linalg.norm((query2),2))
print(costeta2)

cos query 1
[[ 0.81649658]
 [ 0.        ]
 [-0.        ]
 [ 0.70710678]
 [ 0.        ]]
cos query 2
[[0.57735027]
 [0.        ]
 [0.        ]
 [0.5       ]
 [0.        ]]


## linear correlation

In [ ]:
x  = np.linspace(1,10,10).reshape(10,1) 
y1 = 0.01*x  + 1e-2*np.random.randn(10,1)
y2 = (0.01+1e-1*np.random.rand(10,1))* x + 1e-1*np.random.rand(10,1)
y3 = np.linspace(10,1,10).reshape(10,1) + 1e-1*np.random.randn(10,1)
print( np.hstack((x,y1,y2,y3)))

x=x/np.linalg.norm(x,2)
y1=y1/np.linalg.norm(y1,2)
y2=y2/np.linalg.norm(y2,2)
y3=y3/np.linalg.norm(y3,2)
m = 10
print( np.hstack((x,y1,y2,y3)))

In [236]:
#linear correlation

#mean of the data
meanx=np.mean(x)
#standard deviation of data
e=np.ones((m,1))
sigmax=(np.sqrt(np.sum((x- meanx*e)**2,0)))/np.sqrt(m)

In [237]:
zx = (x-meanx*e)/sigmax

In [238]:
meany1 = np.mean(y1)
sigmay1 = np.linalg.norm(y1-meany1*e,2)/np.sqrt(m)
zy1 = (y1-meany1*e)/sigmay1


meany2 = np.mean(y2)
sigmay2 = np.linalg.norm(y2-meany2*e,2)/np.sqrt(m)
zy2 = (y2-meany2*e)/sigmay2

meany3 = np.mean(y3)
sigmay3 = np.linalg.norm(y3-meany3*e,2)/np.sqrt(m)
zy3 = (y3-meany3*e)/sigmay3

In [239]:
np.dot(zy1.T,zx)/m

array([[0.95566607]])

In [240]:
np.dot(zy2.T,zx)/m

array([[0.78412982]])

In [241]:
np.dot(zy3.T,zx)/m

array([[-0.99949805]])